In [1]:
import pandas as pd

# 1. Ler a base de dados de funcionários (arquivo CSV)
employees_df = pd.read_csv('funcionarios.csv')
# Calcular capacidade (número de funcionários disponíveis) por setor
sectors = ["A", "B", "C", "D"]
capacity = {sec: 0 for sec in sectors}
for sec in sectors:
    # Conta funcionários que possuem o setor em sua lista de habilidades
    capacity[sec] = employees_df['Setores'].str.contains(sec).sum()
print("Capacidade de funcionários por setor:", capacity)

# 2. Definir as tarefas com horas necessárias em cada setor (exemplo fictício)
tasks = [
    {"Tarefa": "T1", "A": 20, "B": 10, "C": 5,  "D": 0},   # T1 precisa 20h em A, 10h em B, 5h em C, nada em D
    {"Tarefa": "T2", "A": 5,  "B": 5,  "C": 5,  "D": 5},   # T2 precisa 5h em cada setor
    {"Tarefa": "T3", "A": 0,  "B": 10, "C": 10, "D": 10},  # T3 não precisa de A (0h), mas tem B, C, D
    {"Tarefa": "T4", "A": 10, "B": 0,  "C": 20, "D": 30},  # T4 não precisa de B, mas tem A, C, D (D é grande)
    {"Tarefa": "T5", "A": 15, "B": 15, "C": 15, "D": 15}   # T5 precisa de 15h em cada setor
]

# Preparar estado inicial das tarefas
for task in tasks:
    # restante de horas por setor (cópia dos valores iniciais)
    task["restante"] = {sec: task[sec] for sec in sectors}
    # definir estágio atual (primeiro setor que tenha horas > 0)
    for sec in sectors:
        if task[sec] > 0:
            task["estagio"] = sec
            break
    else:
        task["estagio"] = None  # nenhuma hora necessária em nenhum setor

# 3. Simular alocação de recursos dia a dia
dia = 0
while True:
    # Verificar se todas as tarefas terminaram
    tarefas_pendentes = [t for t in tasks if t["estagio"] is not None]
    if not tarefas_pendentes:
        break  # sai do loop se não há tarefas pendentes (todas concluídas)
    dia += 1

    # Conjuntos para acompanhar quem trabalhou em que setor no dia (evita dupla alocação do mesmo funcionário em setores diferentes no mesmo dia)
    # (Simplificação: assumimos que um funcionário só trabalha em um setor por dia)
    setores_ocupados = {sec: 0 for sec in sectors}  # quantos funcionários usados em cada setor no dia

    # Tentar alocar cada setor (A, B, C, D) neste dia
    for sec in sectors:
        # Lista de tarefas atualmente nesse estágio `sec`
        tarefas_no_setor = [t for t in tasks if t["estagio"] == sec]
        if not tarefas_no_setor:
            continue  # nenhuma tarefa neste setor neste dia

        # Ordenar tarefas por horas restantes nesse setor (pode usar outra prioridade se desejado)
        tarefas_no_setor.sort(key=lambda x: x["restante"][sec])
        for task in tarefas_no_setor:
            # Se ainda há capacidade disponível no setor `sec` neste dia, alocar um funcionário para esta tarefa
            if setores_ocupados[sec] < capacity[sec]:
                # Horas de trabalho em um dia (considerando jornada de 8h)
                horas_trabalhadas = 8 
                # Deduzir as horas trabalhadas do restante da tarefa naquele setor
                task["restante"][sec] = max(0, task["restante"][sec] - horas_trabalhadas)
                setores_ocupados[sec] += 1  # um funcionário alocado no setor

                # Se a tarefa concluiu as horas desse setor, avançar para o próximo setor
                if task["restante"][sec] == 0:
                    # Descobrir próximo setor necessário (que tenha horas > 0)
                    prox_estagio = None
                    for nxt in sectors:
                        if nxt == sec:
                            # pular o próprio e os anteriores
                            continue
                        if task["restante"].get(nxt, 0) and task["restante"][nxt] > 0:
                            prox_estagio = nxt
                            break
                    task["estagio"] = prox_estagio
                    if prox_estagio is None:
                        # tarefa finalizada completamente
                        task["conclusao"] = dia
            else:
                # Capacidade esgotada no setor, não pode alocar mais tarefas aqui neste dia
                break

# 4. Resultado: duração total e término de cada tarefa
duracao_total = dia
print(f"Projeto concluído em {duracao_total} dias.")
for task in tasks:
    print(f"Tarefa {task['Tarefa']} terminada no dia {task.get('conclusao', dia)}.")


Capacidade de funcionários por setor: {'A': 43, 'B': 37, 'C': 36, 'D': 6}
Projeto concluído em 7 dias.
Tarefa T1 terminada no dia 4.
Tarefa T2 terminada no dia 1.
Tarefa T3 terminada no dia 4.
Tarefa T4 terminada no dia 7.
Tarefa T5 terminada no dia 5.
